### This notebook compares the ML-forecast model with the numerical model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing the essential libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K 
import xarray as xr
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K

### Setting up the path

In [ ]:
original = "/content/drive/MyDrive/Lightning/data/data_comparison/ground_truth.nc"
predicted = "/content/drive/MyDrive/Lightning/data/data_comparison/ml_model.nc"
numerical = "/content/drive/MyDrive/Lightning/data/data_comparison/numerical_model.nc"

### Defing the metrics used for the evaluation of he model.

In [ ]:
def ETS(y_true, y_pred):
    ytrue = y_true
    ypred = y_pred
    N1 = np.sum((ypred > 0) & (ytrue > 0))
    N2 = np.sum((ypred > 0) & (ytrue < 1))
    N3 = np.sum((ypred < 1) & (ytrue > 0))
    N4 = np.sum((ypred < 1) & (ytrue < 1))
    R = ((N1+N2)*(N1+N3))/(N1+N2+N3+N4)
    ets = (N1-R)/((N1+N2+N3)-R)
    return ets

def POD(y_true, y_pred):
    ytrue = y_true
    ypred = y_pred
    true_positives = K.sum(ytrue * ypred)
    possible_positives = K.sum(ytrue)
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def FAR(y_true, y_pred):
    ytrue = y_true
    ypred = y_pred
    true_positives = K.sum(ytrue * ypred)
    predicted_positives = K.sum(ypred)
    precision = true_positives / (predicted_positives + K.epsilon())
    return 1 - precision

### Reading the data

In [ ]:
original_data = xr.open_dataset(original)['Lightning'].values
predicted_data = xr.open_dataset(predicted)['Lightning'].values
numerical_data = xr.open_dataset(numerical)['Lightning'].values

In [ ]:
original_data = np.reshape(original_data, (29*33))
predicted_data = np.reshape(predicted_data, (29*33))
numerical_data = np.reshape(numerical_data, (29*33))

original_data = np.float32(original_data)
numerical_data = np.float32(numerical_data)

### Calculating the metrics

In [ ]:
pod_1 = POD(y_true=original_data, y_pred=predicted_data)
far_1 = FAR(y_true=original_data, y_pred=predicted_data)
ets_1 = ETS(y_true=original_data, y_pred=predicted_data)

pod_2 = POD(y_true=original_data, y_pred=numerical_data)
far_2 = FAR(y_true=original_data, y_pred=numerical_data)
ets_2 = ETS(y_true=original_data, y_pred=numerical_data)

In [ ]:
print('Numerical calculation- POD: {pod}, FAR: {far}, ETS: {ets}'.format(pod=pod_2, far=far_2, ets=ets_2))
print('ML prediction--------- POD: {pod}, FAR: {far}, ETS: {ets}'.format(pod=pod_1, far=far_1, ets=ets_1))

Numerical calculation- POD: 0.1492537260055542, FAR: 0.7297297716140747, ETS: 0.08105945426902457
ML prediction--------- POD: 0.7313432693481445, FAR: 0.25757575035095215, ETS: 0.5590790616854909
